In [107]:
import boto3
import os
import json
from dynamodb_json import json_util as dyjson
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import plotly.express as px 
from boto3.dynamodb.conditions import Key

In [108]:
ROOT_DIR = os.path.dirname(os.path.abspath('Strava_Streams.ipynb'))

json_file = open(os.path.join(ROOT_DIR, 'strava_config1.json'), 'r')
json_str = json_file.read()

s3_info = json.loads(json_str)['s3_credentials']

# AWS s3 Credentials
bucket_name = s3_info['bucket_name']
access_key = s3_info['access_key']
secret_access_key = s3_info['secret_access_key']

with open('mapbox_tkn.txt', 'r') as f: 
    mapbox_key = f.read().strip()

In [109]:
client = boto3.client(
        'dynamodb',
        aws_access_key_id = access_key,
        aws_secret_access_key = secret_access_key,
        region_name='us-west-2'
    )

response = client.query(
    TableName='strava_streams', 
    KeyConditionExpression='#S = :partition AND #T > :date',
    ExpressionAttributeNames={
        "#S": "partition", "#T": "date"
    },
    ExpressionAttributeValues={
        ":partition": {"S": "1"},
        ":date": {"S": "2016-03-20 23:03:03+00:00"}
    }
)

In [110]:
data=dyjson.loads(response)

In [111]:
items = data.get('Items')

In [112]:
df = pd.DataFrame()
for obj in items:
    df2 = pd.DataFrame(obj)
    df = df.append(df2, sort=False)
df

,date,distance,altitude,latlng,time,partition,id,name,manual_entry,type
0,2016-05-12 03:44:17+00:00,0.0,68.2,"[61.076166, -149.777041]",0,1,607726565,Evening Run,False,Run
1,2016-05-12 03:44:17+00:00,0.0,69.4,"[61.076166, -149.777041]",4,1,607726565,Evening Run,False,Run
2,2016-05-12 03:44:17+00:00,4.9,70.6,"[61.076166, -149.777041]",9,1,607726565,Evening Run,False,Run
3,2016-05-12 03:44:17+00:00,18.9,71.8,"[61.076166, -149.777041]",14,1,607726565,Evening Run,False,Run
4,2016-05-12 03:44:17+00:00,40.5,72.2,"[61.076166, -149.777041]",21,1,607726565,Evening Run,False,Run
...,...,...,...,...,...,...,...,...,...,...
1853,2016-07-02 21:44:33+00:00,22793.2,82.6,"[60.172419, -149.412125]",8536,1,632699130,Afternoon Run,False,Run
1854,2016-07-02 21:44:33+00:00,22807.0,81.2,"[60.17245, -149.411884]",8539,1,632699130,Afternoon Run,False,Run
1855,2016-07-02 21:44:33+00:00,22830.6,80.8,"[60.172485, -149.411462]",8545,1,632699130,Afternoon Run,False,Run
1856,2016-07-02 21:44:33+00:00,22842.9,82.4,"[60.1725, -149.411241]",8548,1,632699130,Afternoon Run,False,Run


In [113]:
df = df.dropna()
df[["lat", "lng"]]= pd.DataFrame(df['latlng'].values.tolist(), index = df.index)

In [114]:
df

,date,distance,altitude,latlng,time,partition,id,name,manual_entry,type,lat,lng
0,2016-05-12 03:44:17+00:00,0.0,68.2,"[61.076166, -149.777041]",0,1,607726565,Evening Run,False,Run,61.076166,-149.777041
1,2016-05-12 03:44:17+00:00,0.0,69.4,"[61.076166, -149.777041]",4,1,607726565,Evening Run,False,Run,61.076166,-149.777041
2,2016-05-12 03:44:17+00:00,4.9,70.6,"[61.076166, -149.777041]",9,1,607726565,Evening Run,False,Run,61.076166,-149.777041
3,2016-05-12 03:44:17+00:00,18.9,71.8,"[61.076166, -149.777041]",14,1,607726565,Evening Run,False,Run,61.076166,-149.777041
4,2016-05-12 03:44:17+00:00,40.5,72.2,"[61.076166, -149.777041]",21,1,607726565,Evening Run,False,Run,61.076166,-149.777041
...,...,...,...,...,...,...,...,...,...,...,...,...
1853,2016-07-02 21:44:33+00:00,22793.2,82.6,"[60.172419, -149.412125]",8536,1,632699130,Afternoon Run,False,Run,60.172419,-149.412125
1854,2016-07-02 21:44:33+00:00,22807.0,81.2,"[60.17245, -149.411884]",8539,1,632699130,Afternoon Run,False,Run,60.172450,-149.411884
1855,2016-07-02 21:44:33+00:00,22830.6,80.8,"[60.172485, -149.411462]",8545,1,632699130,Afternoon Run,False,Run,60.172485,-149.411462
1856,2016-07-02 21:44:33+00:00,22842.9,82.4,"[60.1725, -149.411241]",8548,1,632699130,Afternoon Run,False,Run,60.172500,-149.411241


In [116]:
fig = px.line_mapbox(df, lat="lat", lon="lng", color = "date")
fig.update_layout(mapbox_accesstoken= mapbox_key)
fig.show()

In [ ]:
client = boto3.client(
        'dynamodb',
        aws_access_key_id = access_key,
        aws_secret_access_key = secret_access_key,
        region_name='us-west-2'
    )

In [10]:
response = client.get_item(
    TableName='strava_streams',
    Key={'id': {'N': '3068703155'}}
)

data=dyjson.loads(response)
df = pd.DataFrame(data.get('Item'))
df[['lat', 'lng']]= pd.DataFrame(df['latlng'].values.tolist(), index = df.index)
df = df.drop(['latlng'], axis=1)
print(df)

     time  distance          id  altitude        lat         lng
0       0       0.0  3068703155    1513.2  45.664934 -111.049939
1       1       2.6  3068703155    1512.2  45.664945 -111.049909
2       4      15.2  3068703155    1511.4  45.665071 -111.049829
3       6      24.7  3068703155    1511.0  45.665149 -111.049763
4      12      48.3  3068703155    1510.4  45.665257 -111.049494
..    ...       ...         ...       ...        ...         ...
573  3085   10561.8  3068703155    1498.2  45.665175 -111.048407
574  3090   10583.3  3068703155    1498.0  45.665150 -111.048682
575  3095   10604.5  3068703155    1498.2  45.665123 -111.048950
576  3100   10626.2  3068703155    1498.6  45.665111 -111.049228
577  3104   10637.8  3068703155    1499.0  45.665118 -111.049376

[578 rows x 6 columns]


In [11]:
fig = px.line_mapbox(df, lat="lat", lon="lng")
fig.update_layout(mapbox_style="dark",
                  mapbox_accesstoken= mapbox_key)
fig.show()